1. Project Title and Introduction

Book Recommender System using Sentence Transformers

This project builds a content-based book recommendation system using book titles and authors. It uses Sentence Transformers to generate embeddings and cosine similarity to suggest similar books.


2. Install & Import Libraries

In [ ]:
!pip install sentence-transformers

import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 66.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 775.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

3. Cell 3: Upload and Load Dataset

In [ ]:
from google.colab import files

# Upload books.csv from Goodbooks-10k
uploaded = files.upload()

# Load into pandas
df = pd.read_csv("books.csv")

# Combine title and author as text
book_texts = (df['title'] + ' by ' + df['authors']).tolist()
book_texts = book_texts[:100]  # use a smaller set for quick testing


Saving books.csv to books.csv


4. Load the Dataset into a DataFrame

In [ ]:
# Load the Dataset into a DataFrame
import pandas as pd

# Read the CSV file
df = pd.read_csv("books.csv")

# Preview the data
print(df.columns)
df.head()

Index(['id', 'book_id', 'best_book_id', 'work_id', 'books_count', 'isbn',
       'isbn13', 'authors', 'original_publication_year', 'original_title',
       'title', 'language_code', 'average_rating', 'ratings_count',
       'work_ratings_count', 'work_text_reviews_count', 'ratings_1',
       'ratings_2', 'ratings_3', 'ratings_4', 'ratings_5', 'image_url',
       'small_image_url'],
      dtype='object')


,id,book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,...,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,...,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,...,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...


5. Generate Embeddings


In [ ]:
# Load transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate embeddings
embeddings = model.encode(book_texts)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

6. Recommend Similar Books

In [ ]:
# Pick one book (e.g., the first)
target_index = 0
target_embedding = [embeddings[target_index]]

# Similarity calculation
similarities = cosine_similarity(target_embedding, embeddings)[0]
similar_indices = similarities.argsort()[::-1][1:6]

print(f"📘 Target Book: {book_texts[target_index]}\n")
print("📚 Top 5 Similar Books:")
for i in similar_indices:
    print(f"- {book_texts[i]} (Similarity: {similarities[i]:.2f})")

📘 Target Book: The Hunger Games (The Hunger Games, #1) by Suzanne Collins

📚 Top 5 Similar Books:
- Catching Fire (The Hunger Games, #2) by Suzanne Collins (Similarity: 0.91)
- Mockingjay (The Hunger Games, #3) by Suzanne Collins (Similarity: 0.91)
- City of Bones (The Mortal Instruments, #1) by Cassandra Clare (Similarity: 0.57)
- The Giver (The Giver, #1) by Lois Lowry (Similarity: 0.53)
- The Perks of Being a Wallflower by Stephen Chbosky (Similarity: 0.53)


7. Optional — Wrap It as a Function

In [ ]:
def recommend_books(index, texts, model, top_n=5):
    embeddings = model.encode(texts)
    target_embedding = [embeddings[index]]
    similarities = cosine_similarity(target_embedding, embeddings)[0]
    similar_indices = similarities.argsort()[::-1][1:top_n+1]

    print(f"📘 Target Book: {texts[index]}\n")
    print("📚 Top Recommendations:")
    for i in similar_indices:
        if i != index:
            print(f"- {texts[i]} (Similarity: {similarities[i]:.2f})")

# Try it
recommend_books(0, book_texts, model)

📘 Target Book: The Hunger Games (The Hunger Games, #1) by Suzanne Collins

📚 Top Recommendations:
- Catching Fire (The Hunger Games, #2) by Suzanne Collins (Similarity: 0.91)
- Mockingjay (The Hunger Games, #3) by Suzanne Collins (Similarity: 0.91)
- City of Bones (The Mortal Instruments, #1) by Cassandra Clare (Similarity: 0.57)
- The Giver (The Giver, #1) by Lois Lowry (Similarity: 0.53)
- The Perks of Being a Wallflower by Stephen Chbosky (Similarity: 0.53)


8. Conclusion


In [ ]:
#Conclusion
#This recommender system uses sentence embeddings on book titles and authors to find similar books. It's a simple but effective content-based approach. For future improvements, we can include book descriptions or user ratings.


9. Install Gradio in Colab

In [9]:
!pip install gradio --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 70.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.8 MB/s eta 0:00:00


 10. Add the Gradio Recommender Function

In [10]:
import gradio as gr

# Define the recommendation function
def recommend_book(book_title):
    if book_title not in df['title'].values:
        return "Book not found."

    idx = df[df['title'] == book_title].index[0]
    embedding = book_embeddings[idx]

    similarities = cosine_similarity([embedding], book_embeddings)[0]
    similar_indices = similarities.argsort()[::-1][1:6]  # Top 5 excluding the selected book

    recommendations = df.iloc[similar_indices]['title'].tolist()
    return "\n".join(recommendations)

11. Create the Gradio Interface

In [11]:
book_list = df['title'].dropna().unique().tolist()  # All unique book titles

interface = gr.Interface(
    fn=recommend_book,
    inputs=gr.Dropdown(choices=book_list, label="Choose a book"),
    outputs="text",
    title="📚 Book Recommender",
    description="Pick a book title and get top 5 recommended books"
)

interface.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://db536d65dceb964e75.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
